In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
from ipywidgets import *
import numpy as np
import k3d
import matplotlib.pyplot as plt
from sloppy.optic import *
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.utils import *
from sloppy.tools import *
from sloppy.metrics import *

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\joptic.py:59: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [3]:
from cavities import *

In [4]:
#cavity_parameter_interaction_factory(LensCavQuartic, 'lens_dist', 1e-1, N=100)

In [5]:
cavfct = LensCavQuartic
parname = 'lens_dist'
waists_vs_param(cavfct, parname, 1e-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

25.79675585284281

In [6]:
res = degeneracy_length(LensCavQuartic, 'lens_dist', 1e-1)
Ldeg = res.x
print(res)

     fun: 7977.93568007015
 message: 'Solution found.'
    nfev: 9
  status: 0
 success: True
       x: 25.789324718907547


In [7]:
elements = cavfct(**{parname: Ldeg})
sys = RaySystem(elements)

system = sys.abcd
print(system.Ltot)
system.waist_at(0)
x = np.linspace(0, system.Ltot, 500)
ws = system.compute_waists(x)

plt.figure()
plt.plot(x, ws)
plt.show()

125.96287424209984


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plot_metric(metric_opl, cavfct, {parname: Ldeg, 'c4': 0., 'c6': 0.})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
metric_interaction_factory(metric_opl, cavfct, {parname: Ldeg, 'c4': 3.63e-3, 'c6': 1e-4})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'lens_dist': 1, 'c4': -3, 'c6': -4}


interactive(children=(FloatSlider(value=2.578932471890755, description='lens_dist', layout=Layout(height='30px…

In [18]:
fig, ax = plt.subplots()
im = plot_metric_par_vs_r(metric_dev, cavfct, cavpars={parname: Ldeg, 'c4': 1e-3, 'c6': 0.}, rng = (3.2e-3, 4e-3), param='c4', ax=ax)
im.norm.vmin *=1e9
plt.colorbar(im , ax=ax)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fig, ax = plt.subplots()

im = plot_metric_par2d_at_r(metric_dev, cavfct, cavpars={parname: Ldeg, 'c4': 0., 'c6': 0.}, parama='c4', rnga=(3.2e-3, 4e-3), \
                            paramb='c6', rngb=(0, 2e-4), r_at=8., ax=ax)
#im.norm.vmin *=1e9
ax.ticklabel_format(style='sci',scilimits=(-2,2),axis='both')
plt.colorbar(im , ax=ax)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plot_residual_motion(cavfct, cavpars={parname: Ldeg, 'c4': 3.3e-3, 'c6': 0.}, ar=2., br=0.1, s=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
metrics_interaction_factory(cavfct, {parname: Ldeg, 'c4': 3.63e-3, 'c6': 1e-4}, a=6., s=1, rmax=12., slrng=(0, 1.2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'lens_dist': 1, 'c4': -3, 'c6': -4}


interactive(children=(FloatSlider(value=6.0, description='a', layout=Layout(height='30px', width='80%'), max=1…

In [7]:
from scipy.optimize import minimize_scalar, brentq, minimize
from functools import partial
from collections import OrderedDict

In [8]:
def metric_optim(x, metr, cavfct, cavpars, rmax=10., Nplt=100, degmodenum=1):
    cavpars.update({'c4': x[0], 'c6': x[1]})
    rs, ds = metric_over_r(metric=metr, cavfct=cavfct, cavpars=cavpars, rmax=rmax, Nplt=Nplt, degmodenum=degmodenum)
    dr = rs[1]-rs[0]
    return np.abs(np.sum(ds))*dr

In [9]:
fun = partial(metric_optim, metr=metric_var, cavfct=cavfct, cavpars={parname: Ldeg}, rmax=8.)

bounds = [(2.2e-3, 4e-3), (0., 3e-4)]
res = minimize(fun, x0=[3.5e-3, 1.5e-4], bounds=bounds)
res

      fun: 0.007008013002646455
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([1.80192666e-05, 1.28395558e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 63
      nit: 11
   status: 0
  success: True
        x: array([3.09024578e-03, 8.98560979e-05])

In [12]:
plot_metrics(cavfct, cavpars={parname: Ldeg, 'c4': res.x[0], 'c6': res.x[1]}, s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plot_residual_motion_mult(cavfct, cavpars={parname: Ldeg, 'c4': res.x[0], 'c6': res.x[1]}, ar=np.arange(1,12,1), s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
elements = cavfct()
sys = RaySystem(elements)
system = sys.abcd
mu1, mu2 = system.q
waist = system.waist_at(0)[0]
rmu1 = np.linalg.norm(np.real(mu1[:2]))
rmu2 = np.linalg.norm(np.real(mu2[:2]))

In [27]:
sys.screen.r_to_screen_coords

(4,)

In [28]:
ar=np.linspace(1,10,10,endpoint=True)
br=0.
ap=0.
bp=0.

In [29]:
mu = np.real(ar[:,None]*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))

In [30]:
mu.shape

(10, 4)

In [31]:
ray = sys.screen.eigenvectors_to_rays(mu)

In [32]:
ray.shape

(2, 10, 3)

In [12]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(sys.elements):
    plot += el.plot(opacity=0.4)

for i, el in enumerate(sys.elements):
    plot_element_ax(el, plot, length=5.)

plot_rays(ray, plot)
plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [9]:
rays_interaction_factory(LensCavQuartic, 'lens_dist')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='ar', layout=Layout(height='30px', width='80%'), max=…

In [10]:
from scipy.optimize import minimize_scalar, brentq, minimize